# 1-Import Data

In [ ]:
# import numpy as np
# import pandas as pd
# from ipywidgets import GridBox, Layout

import fileselector as fs
from importlib import reload
reload(fs)
fs.load_dependencies()
# Usage
# Replace '/your/start/directory/' with your actual start directory
Path='/home/coder/project/doric_fiberphotometry/Data/knob'

file_selector = fs.FileSelector(Path)
# file_name = file_selector.get_selected_file()


In [ ]:
## After selecting datasets, the respective info will be displayed.
## Then, click "Load Data" button to load the datasets.
isos_df = file_selector.get_isos_df()
grabda_df = file_selector.get_grabda_df()
event_df = file_selector.get_event_df()

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

import dataexplorer as de
from importlib import reload
try:
    import ipympl
    print("ipympl is installed. You can proceed with %matplotlib ipympl.")
    %matplotlib widget
except ImportError:
    print("ipympl is not installed. Please run !pip install ipympl.")
    !pip install ipympl
    %matplotlib widget


reload(de)

de.plot_and_save_seperated(isos_df, grabda_df, event_df,file_selector)

de.plot_and_save(isos_df,grabda_df,event_df,file_selector)



# Knob Events


In [ ]:
import dataexplorer as de
from importlib import reload
import ipywidgets as widgets
from IPython.display import display
reload(de)

rising_edges = event_df['Data'].diff() == 1
rising_edge_df = event_df[rising_edges].reset_index().rename(columns={'index': 'Sample_Number'})
## Calculate the optimal time window
optimal_window = de.find_optimal_time_window(rising_edge_df,Event_type='Knob')
print("optimal window is :",optimal_window)
## Classify events in the rising_edge_df
# classified_events_df = de.classify_events(rising_edge_df,optimal_window,['Init','Success','Fail'])
classified_events_df = de.classify_events(rising_edge_df,optimal_window,Event_type='knob')

print(classified_events_df.head())
print("Event Number: ",len(classified_events_df))


# Data Prepration


##  Use function get_zdFF to calculate z-dF/F

Download photometry_functions.py file from [here](https://github.com/katemartian/Photometry_data_processing "source").

please cite Jove paper:

__Martianova, E., Aronson, S., Proulx, C.D.__ [Multi-Fiber Photometry to Record Neural Activity in Freely Moving Animal.](https://www.jove.com/video/60278/multi-fiber-photometry-to-record-neural-activity-freely-moving). _J. Vis. Exp._ (152), e60278, doi:10.3791/60278 (2019).

[refrence](https://colab.research.google.com/github/katemartian/Photometry_data_processing/blob/master/Photometry_data_processing.ipynb#scrollTo=Lak9o-Hn3QQW)


In [ ]:
from importlib import reload
import photometry_functions as pf
reload(pf)
# Usage
def save_signal(zdff):
    global signal_df 
    signal_df = zdff
    
    print(signal_df.head())
analysis = pf.PhotometryAnalysis(isos_df,grabda_df,save_signal)



# PSTH Prepration

## Cut and Center the Data

In [ ]:
import photometry_functions as pf
from importlib import reload

reload(pf)

# Example usage:
time_window = (-4, 10)  # For example, 2 seconds before and after the event
event_type = "Init"  # Use "All" to process all events, or specify a particular type like 'Init'
centralized_signals_df_Init = pf.cut_and_center_signals_modified(signal_df, classified_events_df, time_window, event_type)
print(centralized_signals_df_Init.head())

event_type = "Success"  # Use "All" to process all events, or specify a particular type like 'Init'
centralized_signals_df_Success = pf.cut_and_center_signals_modified(signal_df, classified_events_df, time_window, event_type)
print(centralized_signals_df_Success.head())

event_type = "Fail"  # Use "All" to process all events, or specify a particular type like 'Init'
centralized_signals_df_Fail = pf.cut_and_center_signals_modified(signal_df, classified_events_df, time_window, event_type)
print(centralized_signals_df_Fail.head())


## If you want un centralized signals with init you can run this part if not skip this

In [ ]:
# Example usage:
time_window = (-4, 10)  # For example, 2 seconds before and after the event
event_type = "Init"  # Use "All" to process all events, or specify a particular type like 'Init'
centralized_signals_df_Init = pf.cut_and_center_signals(signal_df, classified_events_df, time_window, event_type)
print(centralized_signals_df_Init.head())

event_type = "Success"  # Use "All" to process all events, or specify a particular type like 'Init'
centralized_signals_df_Success = pf.cut_and_center_signals(signal_df, classified_events_df, time_window, event_type)
print(centralized_signals_df_Success.head())

event_type = "Fail"  # Use "All" to process all events, or specify a particular type like 'Init'
centralized_signals_df_Fail = pf.cut_and_center_signals(signal_df, classified_events_df, time_window, event_type)
print(centralized_signals_df_Fail.head())


## Plot Dataset

### Plot all Signals 

In [ ]:
import os
# Plot All signals
y_min = -2
y_max = 4

import photometry_functions as pf
from importlib import reload
reload(pf)





# Init:
fig = plt
pf.plot_cut_signals(centralized_signals_df_Init,y_min,y_max)
# plot_cut_signals(filtered_signal_df)
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/all_Inits.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

fig.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

# Success:
fig = plt
pf.plot_cut_signals(centralized_signals_df_Success,y_min,y_max)
# plot_cut_signals(filtered_signal_df)
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/all_Success.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

fig.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

# Fail:
fig = plt
pf.plot_cut_signals(centralized_signals_df_Fail,y_min,y_max)
# plot_cut_signals(filtered_signal_df)
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/all_Fails.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

fig.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")



### Plot Signals seperatly

In [ ]:


import photometry_functions as pf
from importlib import reload

reload(pf)

# Init
pf.plot_cut_signals_seperated(centralized_signals_df_Init,y_min,y_max)

fig = plt
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/Separeted_Init_Events.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

fig.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

# Success
pf.plot_cut_signals_seperated(centralized_signals_df_Success,y_min,y_max)

fig = plt
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/Separeted_Success_Events.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

fig.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

# Fail
pf.plot_cut_signals_seperated(centralized_signals_df_Fail,y_min,y_max)

fig = plt
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/Separeted_Fail_Events.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

fig.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

### Plot and Select Signals for PSTH

In [ ]:

import photometry_functions as pf
from importlib import reload

reload(pf)

# Example usage:

def handle_filtered_data(df,event_type):
    # Handle the filtered DataFrame here, e.g., save it to a global variable or process it further
    if event_type == 'Init':    
        global filtered_signal_df_Init
        filtered_signal_df_Init = df
        print(filtered_signal_df_Init.head())
    
    if event_type == 'Fail':    
        global filtered_signal_df_Fail
        filtered_signal_df_Fail = df
        print(filtered_signal_df_Fail.head())
    
    if event_type == 'Success':    
        global filtered_signal_df_Success
        filtered_signal_df_Success = df
        print(filtered_signal_df_Success.head())
    
    
    fig = plt
    filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/Selected_Stim_Events_'+event_type+'.png'

    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)

    fig.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Figure saved as {filename}")



# Init:
explorer = pf.SignalExplorer(centralized_signals_df_Init,'Init', save_callback=handle_filtered_data)
# This will display the widgets and allow interaction

# Success:
explorer = pf.SignalExplorer(centralized_signals_df_Success,'Success', save_callback=handle_filtered_data)
# This will display the widgets and allow interaction

# Fail:
explorer = pf.SignalExplorer(centralized_signals_df_Fail,'Fail', save_callback=handle_filtered_data)
# This will display the widgets and allow interaction


## Normalization

In [ ]:
import photometry_functions as pf
from importlib import reload

reload(pf)


# Init:
# Assuming 'Data' is the column in filtered_signal_df that you want to normalize
normalized_signal_df_Init = pf.normalize_signal(filtered_signal_df_Init, column='Data')

# Success:
# Assuming 'Data' is the column in filtered_signal_df that you want to normalize
normalized_signal_df_Success = pf.normalize_signal(filtered_signal_df_Success, column='Data')

# Fail:
# Assuming 'Data' is the column in filtered_signal_df that you want to normalize
normalized_signal_df_Fail = pf.normalize_signal(filtered_signal_df_Fail, column='Data')


# This will normalize between -1 and 1 (not recomended to run it will incrupt the AUC)

In [ ]:
import photometry_functions as pf
from importlib import reload

reload(pf)

# Init:
# Assuming 'Data' is the column in filtered_signal_df that you want to normalize
normalized_signal_df_Init2 = pf.normalize_signal2(normalized_signal_df_Init, column='Data')

# Success:
# Assuming 'Data' is the column in filtered_signal_df that you want to normalize
normalized_signal_df_Success2 = pf.normalize_signal2(normalized_signal_df_Success, column='Data')

# Fail:
# Assuming 'Data' is the column in filtered_signal_df that you want to normalize
normalized_signal_df_Fail2 = pf.normalize_signal2(normalized_signal_df_Fail, column='Data')




# Analyze Resaults

## PSTH 

In [ ]:
import photometry_functions as pf
import os
from importlib import reload

reload(pf)


PSTH_df_Fail=pf.create_psth_with_min_max(normalized_signal_df_Fail)
PSTH_df_Success=pf.create_psth_with_min_max(normalized_signal_df_Success)

PSTH_df_std_Fail = pf.create_psth_with_std(normalized_signal_df_Fail)
PSTH_df_std_Success = pf.create_psth_with_std(normalized_signal_df_Success)

PSTH_df_ci_Fail = pf.create_psth_with_ci(normalized_signal_df_Fail)
PSTH_df_ci_Success = pf.create_psth_with_ci(normalized_signal_df_Success)

# Create a figure and a 3x1 grid of subplots
fig, axs = plt.subplots(3, 1, figsize=(12, 18))


# PSTH_df_Init=pf.create_psth_with_min_max(normalized_signal_df_Init)
# pf.plot_psth(PSTH_df_Init, -3, 3,'black','Init')
pf.plot_psth(PSTH_df_ci_Fail, -3, 3,'black','Fail',pos=axs[2])
pf.plot_psth(PSTH_df_ci_Success, -3, 3,'green','Success',pos=axs[2])


pf.plot_psth(PSTH_df_std_Fail, -3, 3,'black','Fail',pos=axs[1])
pf.plot_psth(PSTH_df_std_Success, -3, 3,'green','Success',pos=axs[1])
# plt.subplots(2,2,2)

pf.plot_psth(PSTH_df_Fail, -3, 3,'black','Fail',pos=axs[0])
pf.plot_psth(PSTH_df_Success, -3, 3,'green','Success',pos=axs[0])

plt.tight_layout()

plt.show()

filename = file_selector.file_path.split('.')[0]+'/Figs/'+'PSTH.png'


directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

fig.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")


## HeatMap

In [ ]:
import photometry_functions as pf
import os
from importlib import reload

reload(pf)

filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/Heatmap_init.png'
directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)


pf.heatmap_plot(normalized_signal_df_Init)
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/Heatmap_Fail.png'
pf.heatmap_plot(normalized_signal_df_Fail)
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

filename = file_selector.file_path.split('.')[0]+'/Figs/'+'/Heatmap_Success.png'
pf.heatmap_plot(normalized_signal_df_Success)
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")


### Area Under Curves 

In [ ]:
import photometry_functions as pf
import fileselector as fs
import os
from importlib import reload
reload(fs)
reload(pf)
fs.load_dependencies()

# Success
plt.figure(figsize=(12, 6))

pf.plot_psth_auc(PSTH_df_std_Success,-2, 3.5,'green','Stim')
plt.show()
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'AUC_Success.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

pf.plot_auc_bars_with_duration(PSTH_df_std_Success,-0.1, 0.1,(-2,0),(0,3))
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'AUC_Bars_Success.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

# Fail
plt.figure(figsize=(12, 6))

pf.plot_psth_auc(PSTH_df_std_Fail,-2, 3.5,'green','Stim')
plt.show()
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'AUC_Fail.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")

pf.plot_auc_bars_with_duration(PSTH_df_std_Fail,-0.1, 0.1,(-2,0),(0,3))
filename = file_selector.file_path.split('.')[0]+'/Figs/'+'AUC_Bars_Fail.png'

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {filename}")



# saving all the Dataframes as CSV

In [ ]:
import pandas as pd
import datetime

def save_dataframes_to_csv():
    # Fetch the current datetime to append to file names to avoid overwriting
    now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = file_selector.file_path.split('.')[0]+'/Data/'+'test.csv"'
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Loop through all global variables
    for var_name, value in list(globals().items()):
        # Check if the value is an instance of pd.DataFrame
        if isinstance(value, pd.DataFrame):
            # Construct filename with a timestamp to prevent overwriting
            file = f"{var_name}_{now}.csv"
            filename = file_selector.file_path.split('.')[0]+'/Data/'+file

            # Save the DataFrame to a CSV file
            value.to_csv(filename, index=False)
            print(f"Saved {filename}")


# Call the function to save all DataFrames
save_dataframes_to_csv()


# reloading saved data

In [ ]:
import pandas as pd

def load_dataframe_from_csv(file_path):
    """
    Loads a DataFrame from a CSV file.

    Parameters:
    file_path (str): The path to the CSV file to be loaded.

    Returns:
    pandas.DataFrame: The DataFrame containing the data from the CSV file.
    """
    # Loading the DataFrame from the specified CSV file
    df = pd.read_csv(file_path)
    
    return df

# Example usage:
# Replace 'your_file.csv' with the path to the CSV file you want to load

csv_file_path = 'your_file.csv'
df = load_dataframe_from_csv(csv_file_path)
# Optionally, you can display the first few rows of the loaded DataFrame
